In [1]:
import avocado
import numpy as np
import pandas as pd

In [2]:
%matplotlib

Using matplotlib backend: module://ipympl.backend_nbagg


# Load a dataset and explore it

In [3]:
d_train = avocado.load('plasticc_train')

In [4]:
d_train.plot_interactive()

interactive(children=(IntSlider(value=0, description='index', max=7847), Dropdown(description='category', opti…

# Load parts of a dataset

In [9]:
d_test_meta = avocado.Dataset.load('plasticc_test', metadata_only=True)

In [10]:
d_test_chunk = avocado.Dataset.load('plasticc_test', chunk=123, num_chunks=1000)

In [8]:
d_test_chunk.plot_interactive()

interactive(children=(IntSlider(value=0, description='index', max=3492), Dropdown(description='category', opti…

# Augmentation tests

## Single augments

In [24]:
d_train_chunk = avocado.Dataset.load('plasticc_train', chunk=1, num_chunks=500)

In [13]:
augmentor = avocado.plasticc.PlasticcAugmentor()

In [15]:
reference_object = d_train_chunk.get_object(3, category=90)
reference_object.plot_light_curve()

FigureCanvasNbAgg()

In [16]:
augmented_object = augmentor.augment_object(reference_object)
print(reference_object.metadata['redshift'])
print(augmented_object.metadata['redshift'])
augmented_object.plot_light_curve()

Failed to generate a light curve for redshift 0.85. Retrying.


0.33899999999999997
0.4036294272977429


FigureCanvasNbAgg()

## Full dataset augments

In [25]:
d_aug = augmentor.augment_dataset('augment_plasticc_example', d_train_chunk, 1)

100%|██████████| 16/16 [00:03<00:00,  4.35it/s]


## Save and reload the augmented dataset

In [28]:
d_aug.write(overwrite=True)

Overwriting ./data/augment_plasticc_example.h5...


In [31]:
d_aug_load = avocado.Dataset.load('augment_plasticc_example')

# Featurizing tests

## Single featurize

In [50]:
d_example = avocado.Dataset.load('plasticc_train', chunk=0, num_chunks=1000)
d_example.name = 'plasticc_example'

In [51]:
featurizer = avocado.plasticc.PlasticcFeaturizer()

In [52]:
o = d_example.objects[2]
o.plot_light_curve()
featurizer.extract_features(o)

FigureCanvasNbAgg()

{'host_photoz': 0.226,
 'host_photoz_error': 0.016,
 'length_scale': 10.099629214774353,
 'max_mag': -3.8318039858678974,
 'pos_flux_ratio': 0.954620659825224,
 'max_flux_ratio_red': 0.5563387726823994,
 'max_flux_ratio_blue': 0.16964906293178483,
 'min_flux_ratio_red': 0.7359756304138866,
 'min_flux_ratio_blue': 0.47210588103685347,
 'max_dt': 53.0,
 'positive_width': 174.37921761988284,
 'negative_width': 225.37434770566816,
 'time_fwd_max_0.5': 63.0,
 'time_fwd_max_0.2': 90.0,
 'time_fwd_max_0.5_diff_red': 4.0,
 'time_fwd_max_0.5_diff_blue': 0.0,
 'time_fwd_max_0.2_diff_red': 4.0,
 'time_fwd_max_0.2_diff_blue': 0.0,
 'time_bwd_max_0.5': 96.0,
 'time_bwd_max_0.2': 143.0,
 'time_bwd_max_0.5_diff_red': 4.0,
 'time_bwd_max_0.5_diff_blue': -20.0,
 'time_bwd_max_0.2_diff_red': 5.0,
 'time_bwd_max_0.2_diff_blue': 0.0,
 'frac_s2n_5': 0.06666666666666667,
 'frac_s2n_-5': 0.0,
 'frac_background': 0.9121212121212121,
 'time_width_s2n_5': 75.78169999999955,
 'count_max_center': 0,
 'count_max_r

## Featurization of a full dataset

In [53]:
d_example.extract_raw_features(featurizer)
d_example.write_raw_features(overwrite=True)

100%|██████████| 7/7 [00:02<00:00,  3.01it/s]


In [54]:
d_example.select_features(featurizer)

,host_photoz,host_photoz_error,length_scale,max_mag,pos_flux_ratio,max_flux_ratio_red,max_flux_ratio_blue,min_flux_ratio_red,min_flux_ratio_blue,max_dt,...,count_max_rise_100,count_max_fall_20,count_max_fall_50,count_max_fall_100,peak_frac_2,total_s2n,percentile_diff_10_50,percentile_diff_30_50,percentile_diff_70_50,percentile_diff_90_50
object_id,,,,,,,,,,,,,,,,,,,,,
plasticc_000000615,0.000,0.000,0.810587,-7.065665,0.617705,0.408176,0.625225,0.467499,0.618597,-1.0,...,58,27,55,65,0.937435,1680.334319,-3.311567e-01,-2.416353e-01,2.819924e-01,5.269828e-01
plasticc_000000713,1.627,0.255,6.766257,-2.311596,0.450711,0.463458,0.511538,0.453511,0.488457,11.0,...,80,27,52,52,0.937435,74.352059,-4.891333e-01,-2.788420e-01,2.199771e-01,4.096244e-01
plasticc_000000730,0.226,0.016,10.099629,-3.831804,0.954621,0.556339,0.169649,0.735976,0.472106,53.0,...,0,0,21,59,0.937435,62.579015,-1.135296e-01,-6.063487e-02,6.648358e-02,2.860253e-01
plasticc_000000745,0.281,1.152,9.044355,-6.012492,0.967626,0.422034,0.498338,0.410772,0.537993,-7.0,...,0,0,8,38,0.937435,305.326998,-8.720984e-03,-4.099615e-03,1.559579e-02,1.011446e-01
plasticc_000001124,0.241,0.018,9.252276,-5.635482,0.979720,0.476268,0.331214,0.597531,0.612532,18.0,...,0,0,13,46,0.937435,182.567957,-1.740863e-02,-8.369013e-03,3.712748e-02,1.427409e-01
plasticc_000001227,0.000,0.000,13.751474,28.852706,2.638427,0.411026,0.569420,-0.418116,-0.558258,-56.0,...,0,0,0,0,0.937435,36.184640,-1.841864e+12,-9.436215e+11,7.972366e+11,1.941673e+12
plasticc_000001598,0.182,0.030,6.890417,-7.645497,0.994799,0.396718,0.581359,0.762039,0.672671,7.0,...,95,0,0,0,0.937435,534.444428,-2.657124e-03,-1.136119e-03,8.776009e-04,2.648860e-03


In [55]:
d_example.load_raw_features()

,host_specz,host_photoz,host_photoz_error,ra,decl,mwebv,ddf,count,gp_fit_0,gp_fit_1,...,peaks_neg_lsstr_frac_3,peaks_neg_lssti_count,peaks_neg_lssti_frac_2,peaks_neg_lssti_frac_3,peaks_neg_lsstz_count,peaks_neg_lsstz_frac_2,peaks_neg_lsstz_frac_3,peaks_neg_lssty_count,peaks_neg_lssty_frac_2,peaks_neg_lssty_frac_3
object_id,,,,,,,,,,,,,,,,,,,,,
plasticc_000000615,0.000,0.000,0.000,349.0461,-61.9438,0.017,True,352,11.133506,0.810587,...,0.986609,19,0.979853,0.944808,20,0.983326,0.979975,25,0.996985,0.992113
plasticc_000000713,1.818,1.627,0.255,53.0859,-27.7844,0.007,True,350,2.205748,6.766257,...,0.793569,6,0.806771,0.803367,6,0.904351,0.765449,5,0.938004,0.754947
plasticc_000000730,0.232,0.226,0.016,33.5742,-6.5796,0.021,True,330,4.265123,10.099629,...,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
plasticc_000000745,0.304,0.281,1.152,0.1899,-45.5867,0.007,True,351,7.231568,9.044355,...,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
plasticc_000001124,0.193,0.241,0.018,352.7113,-63.8237,0.024,True,352,6.264917,9.252276,...,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
plasticc_000001227,0.000,0.000,0.000,35.6836,-5.3794,0.020,True,330,-30.000000,13.751474,...,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
plasticc_000001598,0.135,0.182,0.030,347.8467,-64.7609,0.019,True,352,10.207869,6.890417,...,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN


In [56]:
d_example.get_raw_features_path()

'./features/features_v1_plasticc_example.h5'

# Other tests

In [3]:
d_test_chunk = avocado.load('plasticc_test', chunk=361, num_chunks=500)

In [10]:
d_test_chunk.plot_light_curve(object_id='plasticc_094389753', verbose=True)

GP fit failed for AstronomicalObject(object_id=plasticc_094389753)! Using guessed GP parameters. This is usually OK.


           object_id: plasticc_094389753
            category: 90
            galactic: False
            redshift: 0.866
          host_specz: -9.0
         host_photoz: 0.9229999999999999
   host_photoz_error: 0.034
                  ra: 310.957
                decl: -20.4236
                 ddf: False
               mwebv: 0.042
       true_submodel: 1
        true_distmod: 43.717
        true_lensdmu: -0.017
           true_vpec: 236.5
             true_rv: 0.0
             true_av: 0.0
        true_peakmjd: 59746.914000000004
       libid_cadence: 47919
             tflux_u: 0.0
             tflux_g: 0.0
             tflux_r: 0.0
             tflux_i: 0.0
             tflux_z: 0.0
             tflux_y: 0.0
OrderedDict([('k1:log_constant', 4.705099522673035), ('k2:metric:log_M_0_0', 5.991464547107982)])
      fun: 483.5321572534723
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-2.48080723e-05,  6.71629555e-06])
  message: b'ABNORMAL_TERMINATION_IN_LNSRC

FigureCanvasNbAgg()

In [4]:
d_aug = avocado.load('plasticc_augment', metadata_only=True)

In [5]:
d_aug.load_raw_features()

,host_specz,host_photoz,host_photoz_error,ra,decl,mwebv,ddf,count,gp_fit_0,gp_fit_1,...,peaks_neg_lsstr_frac_3,peaks_neg_lssti_count,peaks_neg_lssti_frac_2,peaks_neg_lssti_frac_3,peaks_neg_lsstz_count,peaks_neg_lsstz_frac_2,peaks_neg_lsstz_frac_3,peaks_neg_lssty_count,peaks_neg_lssty_frac_2,peaks_neg_lssty_frac_3
object_id,,,,,,,,,,,,,,,,,,,,,
plasticc_000000615,0.0,0.0,0.0,349.0461,-61.9438,0.017000,True,352,11.133506,0.810587,...,0.986609,19,0.979853,0.944808,20,0.983326,0.979975,25,0.996985,0.992113
plasticc_000000615_aug_AfwhmLEKQL,0.0,0.0,0.0,349.0461,-61.9438,0.016118,False,122,6.454652,0.388266,...,0.279241,6,0.648880,0.289609,6,0.642546,0.319762,7,0.640724,0.379671
plasticc_000000615_aug_ArBbGDOTEa,0.0,0.0,0.0,349.0461,-61.9438,0.015992,False,114,9.346215,1.725679,...,0.880168,8,0.867330,0.589852,7,0.850882,0.696972,9,0.854364,0.801094
plasticc_000000615_aug_BSDkftODYF,0.0,0.0,0.0,349.0461,-61.9438,0.017554,False,123,9.070464,1.050112,...,0.855512,7,0.995446,0.929395,9,0.977737,0.777834,9,0.999165,0.700707
plasticc_000000615_aug_BxduDtddOL,0.0,0.0,0.0,349.0461,-61.9438,0.017249,False,145,10.291138,3.412602,...,0.472386,4,0.963368,0.676012,4,0.953678,0.839354,4,0.932417,0.888449
plasticc_000000615_aug_CJbvmPURVz,0.0,0.0,0.0,349.0461,-61.9438,0.016701,False,120,7.998788,0.333521,...,0.838042,4,0.937175,0.880557,4,0.922710,0.905610,4,0.937640,0.908554
plasticc_000000615_aug_CzbxRXPLug,0.0,0.0,0.0,349.0461,-61.9438,0.017332,False,129,9.443647,1.435118,...,0.876341,3,0.993735,0.879445,4,0.985381,0.878300,4,0.978954,0.877283
plasticc_000000615_aug_DljmkHyKdP,0.0,0.0,0.0,349.0461,-61.9438,0.018484,True,200,10.488761,1.261808,...,0.722113,5,0.966933,0.765594,5,0.896089,0.721386,5,0.850428,0.672774
plasticc_000000615_aug_FRrdUOPsxT,0.0,0.0,0.0,349.0461,-61.9438,0.018340,False,111,7.839435,1.914276,...,0.810260,10,0.755604,0.711206,10,0.777022,0.628947,9,0.775035,0.568378


In [8]:
d_aug.select_features(avocado.plasticc.PlasticcFeaturizer())

,host_photoz,host_photoz_error,length_scale,max_mag,pos_flux_ratio,max_flux_ratio_red,max_flux_ratio_blue,min_flux_ratio_red,min_flux_ratio_blue,max_dt,...,count_max_rise_100,count_max_fall_20,count_max_fall_50,count_max_fall_100,peak_frac_2,total_s2n,percentile_diff_10_50,percentile_diff_30_50,percentile_diff_70_50,percentile_diff_90_50
object_id,,,,,,,,,,,,,,,,,,,,,
plasticc_000000615,0.0,0.0,0.810587,-7.065665,0.617705,0.408176,0.625225,0.467499,0.618597,-1.0,...,58,27,55,65,0.474813,1680.334319,-0.331157,-0.241635,0.281992,0.526983
plasticc_000000615_aug_AfwhmLEKQL,0.0,0.0,0.388266,-4.494220,0.544187,0.406372,0.579679,0.404349,0.580693,0.0,...,27,11,29,39,0.474813,13.948564,-0.184616,-0.083996,0.076217,0.223526
plasticc_000000615_aug_ArBbGDOTEa,0.0,0.0,1.725679,-6.208115,0.640410,0.418364,0.578150,0.403252,0.603841,-54.0,...,41,1,1,1,0.474813,21.552022,-0.211926,-0.077323,0.067830,0.496918
plasticc_000000615_aug_BSDkftODYF,0.0,0.0,1.050112,-6.129528,0.621875,0.446448,0.548579,0.488966,0.606091,-17.0,...,50,5,5,5,0.474813,26.031192,-0.308146,-0.096654,0.058491,0.320105
plasticc_000000615_aug_BxduDtddOL,0.0,0.0,3.412602,-7.055409,0.594783,0.426650,0.518387,0.414097,0.600680,0.0,...,44,14,32,32,0.474813,31.118394,-0.200170,-0.043208,0.003737,0.082633
plasticc_000000615_aug_CJbvmPURVz,0.0,0.0,0.333521,-5.233134,0.442099,0.406919,0.572580,0.413330,0.575429,0.0,...,27,6,18,18,0.474813,19.573235,-0.133143,-0.041659,0.028462,0.101829
plasticc_000000615_aug_CzbxRXPLug,0.0,0.0,1.435118,-6.346071,0.611454,0.411038,0.576388,0.410114,0.581818,-92.0,...,12,9,24,58,0.474813,30.598510,-0.083236,-0.036351,0.044520,0.176286
plasticc_000000615_aug_DljmkHyKdP,0.0,0.0,1.261808,-6.888374,0.597957,0.427650,0.556596,0.412243,0.611554,-1.0,...,53,22,49,57,0.474813,35.158022,-0.193342,-0.021838,0.065997,0.418803
plasticc_000000615_aug_FRrdUOPsxT,0.0,0.0,1.914276,-5.158265,0.510406,0.410617,0.577202,0.469533,0.538021,0.0,...,42,6,14,14,0.474813,20.024029,-0.340193,-0.084393,0.073985,0.226262


In [9]:
d_aug.features.sort_index(inplace=True)

In [11]:
d_aug.features.iloc[0]

host_photoz                      0.000000
host_photoz_error                0.000000
length_scale                     0.810587
max_mag                         -7.065665
pos_flux_ratio                   0.617705
max_flux_ratio_red               0.408176
max_flux_ratio_blue              0.625225
min_flux_ratio_red               0.467499
min_flux_ratio_blue              0.618597
max_dt                          -1.000000
positive_width                  44.602053
negative_width                  67.777162
time_fwd_max_0.5                 2.000000
time_fwd_max_0.2                 3.000000
time_fwd_max_0.5_diff_red        0.000000
time_fwd_max_0.5_diff_blue      -1.000000
time_fwd_max_0.2_diff_red        0.000000
time_fwd_max_0.2_diff_blue       0.000000
time_bwd_max_0.5                 3.000000
time_bwd_max_0.2                 5.000000
time_bwd_max_0.5_diff_red        1.000000
time_bwd_max_0.5_diff_blue       1.000000
time_bwd_max_0.2_diff_red        0.000000
time_bwd_max_0.2_diff_blue       0

In [12]:
plt.figure()
plt.hist(d_aug.features[d_aug.metadata['category'] == 90]['time_fwd_max_0.5_ratio_red'], 100, (0.2, 0.8), density=1, alpha=0.5)
plt.hist(d_aug.features[d_aug.metadata['category'] == 15]['time_fwd_max_0.5_ratio_red'], 100, (0.2, 0.8), density=1, alpha=0.5)

FigureCanvasNbAgg()

/home/kboone/packages/conda/lib/python3.7/site-packages/numpy/lib/histograms.py:824: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/home/kboone/packages/conda/lib/python3.7/site-packages/numpy/lib/histograms.py:825: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


(array([0.00000000e+00, 0.00000000e+00, 7.94520983e-03, 0.00000000e+00,
        7.94520983e-03, 0.00000000e+00, 0.00000000e+00, 7.94520983e-03,
        1.58904197e-02, 0.00000000e+00, 2.38356295e-02, 1.58904197e-02,
        7.94520983e-03, 7.94520983e-03, 0.00000000e+00, 7.94520983e-03,
        3.97260492e-02, 2.38356295e-02, 1.58904197e-02, 2.38356295e-02,
        3.17808393e-02, 2.38356295e-02, 4.76712590e-02, 4.76712590e-02,
        2.38356295e-02, 3.97260492e-02, 7.94520983e-02, 7.94520983e-02,
        7.94520983e-02, 6.35616787e-02, 1.35068567e-01, 6.35616787e-02,
        1.50958987e-01, 1.35068567e-01, 1.43013777e-01, 2.62191924e-01,
        3.25753603e-01, 3.41644023e-01, 4.60822170e-01, 6.11781157e-01,
        7.54794934e-01, 1.23945273e+00, 1.35863088e+00, 2.12931624e+00,
        3.05890579e+00, 4.00438576e+00, 5.00548219e+00, 7.21425053e+00,
        1.12424719e+01, 5.23589328e+00, 2.30967250e+01, 1.98312437e+01,
        2.01490521e+01, 1.68200092e+01, 1.21323354e+01, 9.287950

In [12]:
d_aug.plot_interactive()

interactive(children=(IntSlider(value=0, description='index', max=254002), Dropdown(description='category', op…

In [13]:
np.sum(d_aug.metadata['ddf']) / len(d_aug.metadata)

0.0816013984086802

In [15]:
d_aug.features.iloc[0]

host_photoz                       0.000000
host_photoz_error                 0.000000
length_scale                      0.810587
max_mag                          -7.065665
pos_flux_ratio                    0.617705
max_flux_ratio_red                0.408176
max_flux_ratio_blue               0.625225
min_flux_ratio_red                0.467499
min_flux_ratio_blue               0.618597
max_dt                           -1.000000
positive_width                   44.602053
negative_width                   67.777162
time_fwd_max_0.5                  2.000000
time_fwd_max_0.2                  3.000000
time_fwd_max_0.5_ratio_red        0.500000
time_fwd_max_0.5_ratio_blue       0.333333
time_fwd_max_0.2_ratio_red        0.500000
time_fwd_max_0.2_ratio_blue       0.250000
time_bwd_max_0.5                  3.000000
time_bwd_max_0.2                  5.000000
time_bwd_max_0.5_ratio_red        0.571429
time_bwd_max_0.5_ratio_blue       0.571429
time_bwd_max_0.2_ratio_red        0.500000
time_bwd_ma

In [17]:
plt.figure()
plt.hist(d_aug.raw_features['count'], 100)

FigureCanvasNbAgg()

(array([1.0000e+00, 1.4000e+01, 2.8000e+01, 5.2000e+01, 5.3000e+01,
        1.2500e+02, 1.4300e+02, 2.3300e+02, 2.8200e+02, 6.7000e+02,
        6.9400e+02, 1.2880e+03, 1.4940e+03, 2.6930e+03, 2.5900e+03,
        4.6250e+03, 6.1440e+03, 5.9190e+03, 9.6820e+03, 8.6460e+03,
        1.3202e+04, 1.1354e+04, 1.6902e+04, 1.3723e+04, 1.9672e+04,
        1.4952e+04, 1.8882e+04, 1.2883e+04, 1.4290e+04, 8.7260e+03,
        9.6830e+03, 5.9850e+03, 7.1590e+03, 6.4840e+03, 4.2120e+03,
        4.5670e+03, 2.5070e+03, 2.1570e+03, 9.0200e+02, 5.8200e+02,
        2.0700e+02, 2.0800e+02, 1.3200e+02, 2.1100e+02, 1.2700e+02,
        2.0900e+02, 1.3500e+02, 2.1400e+02, 2.2200e+02, 4.0800e+02,
        5.8700e+02, 4.6500e+02, 5.2300e+02, 3.7200e+02, 4.8700e+02,
        4.0300e+02, 5.4800e+02, 4.8800e+02, 6.3400e+02, 3.8000e+02,
        5.0200e+02, 3.7400e+02, 5.0700e+02, 3.6900e+02, 5.0100e+02,
        3.8100e+02, 4.8500e+02, 9.6500e+02, 3.8100e+02, 5.2300e+02,
        3.7900e+02, 4.8500e+02, 4.0100e+02, 4.92